In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 


### Función auxiliar para crear las diferentes conexiones al necesitar navegar por la página.
### Reibe como parametro la url hacía donde va la conexión y se concatena a la URL base.

In [10]:
def abrir_conexion(url):
    url_base="https://www.ubereats.com" ##base
    headers = {'Accept': 'text/html'} ##Encabezado para que acepte la conexión
    url_final=url_base+url ##concatenar direcciones
    pagina=requests.get(url_final,headers=headers) ##Hacer el request a la pag
    if pagina.status_code ==200: ##Control para la conexión
        print("Conexión establecida")
        contenido=BeautifulSoup(pagina.content,"html.parser") ##Bajar html
        return contenido
    else:
        print("Conexión fallido")
        return False
    



### Llamado, apertura y consecución del contenido de la página:

### * Importante, si se desea acceder a la información de otros paises, basta con cambiar '/co/location', por 'xx/location', donde xx representa el dominio de cualquier país. Ejemplo: 'pe/location', para obtener información de Perú.

### Extracción de las diferentes ciudades que tiene UberEats registradas en Colombia, además de la extracción de las urls para navegar allí :

### - A partir de aquí, se trabaja todo con Colombia, dados los parametros de la celda anterior.

### - Se extraen todas las ciudades y las respectivas URL hacía donde dirigen. Esto con el fin de que sea escalable, por si el día de mañana se agregan otras ciudades no presentes en el momento.

In [11]:
divTag=[]
while(len(divTag)==0):
    url = '/co/location' ##Url de Colombia. SOLO INTERESA COLOMBIA.
    contenido=abrir_conexion(url) ##abrir pagina 
    divTag = contenido.find_all("div", {"class": "db dc dd de df dg dh"}) ##Etiqueta donde están almacenados las ciudades del país
ciudades=divTag[0].find_all("span", {"class": "bj bk c3 au"}) ##Obtener las ciudades del país
url_ciudades=divTag[0].find_all("a") ##Obtener las urls que dirigen

Conexión establecida
Conexión establecida


In [12]:
ciudades=[ciudad.text for ciudad in ciudades] ## Almacenar la lista de ciudades que tiene registradas UberEats en Colombia
url_ciudades=[url["href"] for url in url_ciudades] ##Almacenar Urls para cada ciudad

In [13]:
print(ciudades)
print(url_ciudades)

['Armenia', 'Barranquilla', 'Bogotá', 'Bucaramanga', 'Cali - Colombia', 'Manizales', 'Medellín', 'Pereira', 'Santa Marta']
['/co/location/armenia', '/co/location/barranquilla', '/co/location/bogota', '/co/location/bucaramanga', '/co/location/cali', '/co/location/manizales', '/co/location/medellin', '/co/location/pereira', '/co/location/santa-marta']


In [14]:
ciudad_categoria={} ##Guardar las ciudades y las categorias de cada ciudad
ciudad_categoria_url={}

### Tratando de navegar hacía las Urls de las diferentes ciudades Colombianas, almacenar las diferentes categorias que posee. Fue necesario usar el ciclo while, ya que cuando se hacían multiples request seguidos, muchas veces retornaba vacios. Con el while se asegura que repita la petición hasta que obtenga un resultado.

In [15]:
for url in range(0,len(url_ciudades)): ##Iterar sobre las ciudades
    categorias_etiqueta=[] ##auxiliar control
    while len(categorias_etiqueta)==0: ##Control para respuestas vacias por recurrencias
        contenido_ciudad=abrir_conexion(url_ciudades[url]) ##abrir conexión
        categorias_etiqueta=contenido_ciudad.find_all("a", {"class": "ey d8 ez"}) ##Obtener las categorias
        categorias_etiqueta2=contenido_ciudad.find_all("a", {"class": "eq d8 er"}) ##Obtener categorias (alternas)
        categorias_etiqueta+=categorias_etiqueta2 #Unirlas
        

    categorias_ciudad=[etiqueta.text for etiqueta in categorias_etiqueta] ##Nombres de categoria por ciudad
    url_categorias=[etiqueta["href"] for etiqueta in categorias_etiqueta] ## Obtener Url de las caterogias por ciudad
    ciudad_categoria[ciudades[url]]=categorias_ciudad ## Guardar ciudad y respectivas categorías.
    ciudad_categoria_url[ciudades[url]]=url_categorias ## Guardar ciudad y respectivas categorías.



Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida


In [16]:
#ciudad_categoria

### Abrir las conexiones con cada categoria por ciudad y extraer los restaurantes. PARTE CRÍTICA:

In [17]:
ciudad_categoria_definitiva={}

In [18]:
for i in ciudad_categoria_url:
    
    for url in ciudad_categoria_url[i]:
        
        sitio_etiqueta=[]
        while len(sitio_etiqueta)==0:
            contenido_categoria=abrir_conexion(url) ##abrir conexión
            sitio_etiqueta=contenido_categoria.find_all("h3",{"class":"ek bk bl au el al aj em dk"}) 
                             ##Obtener las categorias
        url_restaurante=[]
        for restaurante in sitio_etiqueta:
            url_restaurante.append(restaurante.find_all("a")[0]["href"])

        ciudad_categoria_definitiva[str(i)+"-"+str(url)]=url_restaurante ##Almacenando las lista de restaurantes por ciudad-categoría
        




Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión establecida
Conexión esta

In [19]:

listarestaurantes=[]

### Se accede a cada uno de los diferentes restaurantes, para extraer los atributos requeridos. Posterior a esto se guardan todos como diccionarios, para finalmente llevarlos a un dataFrame.

### El principal problema que se tuvo acá, fue el hecho de que muchos restaurantes varían en su estrcutura HTML, según el estado en el que se encuentren en el momento. Si está abierto, si está cerrado, si es exclusivo de uberEats. Estos factores alteran la estructura y es necesario programar una extraccíón más dinámica.

In [28]:
cont=0
for ciudad_categoriaaux in ciudad_categoria_definitiva:
    for restaurante in ciudad_categoria_definitiva[ciudad_categoriaaux]:
        nombre=[]
        categoria_especifica=[]
        calificacion=[]
        reviews=[]
        direccion=[]
        cont+=1
        print("Conteo: ", cont)
        

        ##ABIERTO Y CERRADO: ###WHILE VALIDA RESPUESTA
        contenido_restaura=abrir_conexion(restaurante) ## Abrir conexión restaurante
        contenido_restaura_aux=contenido_restaura.find("div",{"class":"au at dc dd de df af aq dg"}) #Obtener el nombre, tipo 1
        if contenido_restaura_aux==None:
            contenido_restaura_aux=contenido_restaura.find("div",{"class":"au at d8 d9 da db af aq dc"})
            if contenido_restaura_aux==None:
                print("NADA POR ACAAAAAAAAAAAA: ", restaurante)
        nombre=contenido_restaura_aux.find_all("h1",{"class":"de df dg as"}) #Obtener el nombre, tipo 1
        if len(nombre)==0:
            nombre=contenido_restaura_aux.find_all("h1",{"class":"di dj dk as"}) ##Nombre, alternativo
            if len(nombre)==0:  ##Peor de los casos, no está el nombre
                nombre=contenido_restaura_aux.find_all("h1",{"class":"d4 d5 d6 au"}) ##Nombre, alternativo
                if len(nombre)==0:
                    nombre=""

        categoria_especifica=contenido_restaura_aux.find_all("div",{"class":"c6 cv ba as dh"})  ##Categoria tipo 1
        if len(categoria_especifica)==0:
            categoria_especifica=contenido_restaura_aux.find_all("div",{"class":"c6 cv ba as dl"}) ## Categoria, alternativa 
            if len(categoria_especifica)==0:
                categoria_especifica=contenido_restaura_aux.find_all("div",{"class":"ct cu ck au ft"})
                if len(categoria_especifica)==0:

                    categoria_especifica="" ##Peor de los casos, no hay categoría

        calificacion=contenido_restaura_aux.find_all(lambda x: x.name == 'div' and x.get('class') == ['c4']) ##Calificación

        if len(calificacion)==0: 
            calificacion=contenido_restaura_aux.find_all(lambda x: x.name == 'div' and x.get('class') == ['au'])
            if len(calificacion)==0:
                calificacion=contenido_restaura_aux.find_all(lambda x: x.name == 'div' and x.get('class') == ['as'])
                if len(calificacion)==0:
                    calificacion=contenido_restaura_aux.find_all(lambda x: x.name == 'div' and x.get('class') == ['an'])
                    if(len(calificacion))==0:
                        calificacion=""##Peor de los casos, calificación vacía




            

        reviews=contenido_restaura_aux.find_all(lambda x: x.name == 'div' and x.get('class') == ['dj']) ##Revies tipo 1

        if len(reviews)==0:
            reviews=contenido_restaura_aux.find_all(lambda x: x.name == 'div' and x.get('class') == ['dn']) ##Reviews alternativo
            if len(reviews)==0:
                reviews=contenido_restaura_aux.find_all(lambda x: x.name == 'div' and x.get('class') == ['du']) ##Reviews alternativo
                if len(reviews)==0:

                    reviews="" ##Peor de los casos, reviews está vacío 

        direccion=contenido_restaura_aux.find_all("p",{"class":"c6 cv ba as dm"}) ##Dirección tiene 4 tipos de variaciones, tipo 1
        if len(direccion)==0:
            direccion=contenido_restaura_aux.find_all("p",{"class":"c6 cv ba as dq"}) ##tipo 2
            if len(direccion)==0:
                direccion=contenido_restaura_aux.find_all("p",{"class":"c6 cv ba as dl"}) ## tipo 3
                if len(direccion)==0:
                    direccion=contenido_restaura_aux.find_all("p",{"class":"c6 cv ba as dp"}) ##tipo 4
                    if len(direccion)==0:
                        direccion=contenido_restaura_aux.find_all("p",{"class":"ct cu ck au h8"}) ##tipo 5
                        if len(direccion)==0:

                            direccion=""  ##Peor de los casos, está vacía
        if (nombre != ""):

            nombre=nombre[0].text
            #print(nombre)
        if (categoria_especifica != ""):
            categoria_especifica=categoria_especifica[0].text
            #print(categoria_especifica)
        if (calificacion != ""):
            try:
                print(calificacion)
                calificacion=calificacion[0].text
                
            except:
                print("Calificación:",calificacion)
                calificacion=""
            #print(calificacion)
        if (reviews != ""):
            reviews=reviews[0].text
            #print(reviews)
        if (direccion != ""):
            direccion=direccion[0].text
            #print(direccion)
        print("-------------------------||||||||||||||||||||||||||||||-------------------------")
    #print(nombre[0].text)
    #print(categoria_especifica[0].text)
    #print(calificacion[0].text)
    #print(reviews[0].text)
    #print(direccion[0].text)
        #print(nombre)
        #print(categoria_especifica)
        #print(calificacion)
        #print(reviews)
        #print(direccion)
        #print(nombre[0].text)
        #print(categoria_especifica[0].text)
        #print(calificacion[0].text)
        #print(reviews[0].text)
        #print(direccion[0].text)

        restaux={"Nombre":nombre,"Categoria_Especifica":categoria_especifica,"Calificacion":calificacion,"Reviews":reviews,"Direccion":direccion, "Url":"https://www.ubereats.com"+restaurante}
        listarestaurantes.append(restaux)
        print(restaux)


        ############CUIDADO QUE YA HAY INDICIOS DE PORQUÉ SE VA SIEMPRE POR LA CONDICIÓN. POR ESA CLASE DE MIERDA. OTRA CLASE CON NOMBRE SIMILAR, LLENA DE MIERDA.
    

o Cuba, Pereira, Risaralda 680004 Colombia, \xa0•\xa0Más información', 'Url': 'https://www.ubereats.com/co/pereira/food-delivery/salsabor/tFrsELouQZafnTm9s0CtUQ'}
Conteo:  2356
Conexión establecida
-------------------------||||||||||||||||||||||||||||||-------------------------
{'Nombre': 'Frijoles & Mondongos (Unicentro Pereira)', 'Categoria_Especifica': '$ • Colombiana', 'Calificacion': '', 'Reviews': '', 'Direccion': 'Av. 30 De Agosto # 75 - 51, 576, Pereira, Risaralda 660017 Colombia, CENTRO COMERCIAL UNICENTRO , Risaralda 110221\xa0•\xa0Más información', 'Url': 'https://www.ubereats.com/co/pereira/food-delivery/frijoles-%26-mondongos-unicentro-pereira/NFaTlrmuR0-cD9OW8G43Rg'}
Conteo:  2357
Conexión establecida
[<div class="c4">4.6</div>]
-------------------------||||||||||||||||||||||||||||||-------------------------
{'Nombre': 'Arroz Paisa el Exitaso del Sabor', 'Categoria_Especifica': '$ • Colombiana', 'Calificacion': '4.6', 'Reviews': '(13)', 'Direccion': 'Cl. 34a ##28-28, Pere

In [29]:
restaurantes=pd.DataFrame(listarestaurantes)

In [30]:
restaurantes.head(40)

,Nombre,Categoria_Especifica,Calificacion,Reviews,Direccion,Url
0,Versus,$$ • Comida rápida,4.3,(34),"Carrera 13# 8n -39, Quindío 630003 • Más infor...",https://www.ubereats.com/co/armenia/food-deliv...
1,Don Jacobo Postres y Ponques - Armenia,$ • Postres,4.8,(52),"Cra. 14 No. 9 Norte-69, Av. Bolivar, Armenia, ...",https://www.ubereats.com/co/armenia/food-deliv...
2,Le Ricette Pizza,$$ • Italiana,,,"630001 Cra. 6 #13n-27 Local, Armenia, Quindío ...",https://www.ubereats.com/co/armenia/food-deliv...
3,Versus,$$ • Comida rápida,4.3,(34),"Carrera 13# 8n -39, Quindío 630003 • Más infor...",https://www.ubereats.com/co/armenia/food-deliv...
4,Don Jacobo Postres y Ponques - Armenia,$ • Postres,4.8,(52),"Cra. 14 No. 9 Norte-69, Av. Bolivar, Armenia, ...",https://www.ubereats.com/co/armenia/food-deliv...
5,Le Ricette Pizza,$$ • Italiana,,,"630001 Cra. 6 #13n-27 Local, Armenia, Quindío ...",https://www.ubereats.com/co/armenia/food-deliv...
6,BoticaSol,$$ • Saludable,,,"Cl. 5 Nte. #14–06, Armenia, Quindío 630004 • M...",https://www.ubereats.com/co/armenia/food-deliv...
7,Baronesa,$$ • Latina fusión,4.8,(37),"Armenia, Quindío, Colombia, Mall Del Rio, • M...",https://www.ubereats.com/co/armenia/food-deliv...
8,Popsy - Portal Quindio,$ • Helado y yogur helado,4.8,(37),Centro Comercial Portal De Quindio Avenida 14 ...,https://www.ubereats.com/co/armenia/food-deliv...
9,Popsy - Unicentro Armenia,$ • Helado y yogur helado,4.7,(70),"Unicentro, Armenia, Quindío 630001 • Más infor...",https://www.ubereats.com/co/armenia/food-deliv...


In [4]:
restaurantes.to_csv('restaurantes_08062020pm.csv', encoding='utf-8',index=False)

NameError: name 'restaurantes' is not defined

In [ ]:
import json
json = json.dumps(ciudad_categoria_definitiva)
f = open("definitivo_ciudad_categoria_chile.json","w")
f.write(json)
f.close()

In [ ]:
contenido_restaura=abrir_conexion('/cl/antofagasta/food-delivery/la-ciudad-nueva/G-geeH24TbaDx5BNOxntTQ') ## Abrir conexión restaurante
contenido_restaura_aux=contenido_restaura.find("div",{"class":"au at dc dd de df af aq dg"}) #Obtener el nombre, tipo 1
if contenido_restaura_aux==None:
    contenido_restaura_aux=contenido_restaura.find("div",{"class":"au at d8 d9 da db af aq dc"})




nombre=contenido_restaura.find_all("h1",{"class":"de df dg as"}) #Obtener el nombre, tipo 1
if len(nombre)==0:
    nombre=contenido_restaura.find_all("h1",{"class":"di dj dk as"}) ##Nombre, alternativo
    if len(nombre)==0:
          ##Peor de los casos, no está el nombre
        nombre=contenido_restaura.find_all("h1",{"class":"d4 d5 d6 au"}) ##Nombre, alternativo
        if len(nombre)==0:
            nombre=""

categoria_especifica=contenido_restaura.find_all("div",{"class":"c6 cv ba as dh"})  ##Categoria tipo 1
if len(categoria_especifica)==0:
    categoria_especifica=contenido_restaura.find_all("div",{"class":"c6 cv ba as dl"}) ## Categoria, alternativa 
    if len(categoria_especifica)==0:
        categoria_especifica=contenido_restaura.find_all("div",{"class":"ct cu ck au ft"})
        if len(categoria_especifica)==0:
            categoria_especifica="" ##Peor de los casos, no hay categoría

calificacion=contenido_restaura.find_all(lambda x: x.name == 'div' and x.get('class') == ['c4']) ##Calificación
#soup.find_all(lambda tag: tag.name == 'li' and tag.get('class') == ['z'])
if len(calificacion)==0:
     calificacion=contenido_restaura.find_all(lambda x: x.name == 'div' and x.get('class') == ['au'])
     if len(calificacion)==0:
         calificacion=contenido_restaura.find_all(lambda x: x.name == 'div' and x.get('class') == ['as'])
         if len(calificacion)==0:

             calificacion=contenido_restaura.find_all(lambda x: x.name == 'div' and x.get('class') == ['an'])
             if(len(calificacion))==0:
                 calificacion=""##Peor de los casos, calificación vacía

#css_soup.find_all("p", class_="body strikeout"


            

reviews=contenido_restaura.find_all(lambda x: x.name == 'div' and x.get('class') == ['dj']) ##Revies tipo 1
if len(reviews)==0:
    reviews=contenido_restaura.find_all(lambda x: x.name == 'div' and x.get('class') == ['dn']) ##Reviews alternativo
    if len(reviews)==0:
        reviews=contenido_restaura.find_all(lambda x: x.name == 'div' and x.get('class') == ['du']) ##Reviews alternativo
        if len(reviews)==0:
            reviews="" ##Peor de los casos, reviews está vacío 

direccion=contenido_restaura.find_all("p",{"class":"c6 cv ba as dm"}) ##Dirección tiene 4 tipos de variaciones, tipo 1
if len(direccion)==0:
    direccion=contenido_restaura.find_all("p",{"class":"c6 cv ba as dq"}) ##tipo 2
    if len(direccion)==0:
        direccion=contenido_restaura.find_all("p",{"class":"c6 cv ba as dl"}) ## tipo 3
        if len(direccion)==0:
            direccion=contenido_restaura.find_all("p",{"class":"c6 cv ba as dp"}) ##tipo 4
            if len(direccion)==0:
                direccion=contenido_restaura.find_all("p",{"class":"ct cu ck au h8"}) ##tipo 5
                if len(direccion)==0:
                    direccion=""  ##Peor de los casos, está vacía
if (nombre != ""):
    nombre=nombre[0].text
            #print(nombre)
if (categoria_especifica != ""):
    categoria_especifica=categoria_especifica[0].text
            #print(categoria_especifica)
if (calificacion != ""):
    try:

        print(calificacion)
        calificacion=calificacion[0].text
                
    except:

        print("Calificación:",calificacion)
        calificacion=""
            #print(calificacion)
if (reviews != ""):
    reviews=reviews[0].text
            #print(reviews)
if (direccion != ""):
    direccion=direccion[0].text
            #print(direccion)
print("-------------------------||||||||||||||||||||||||||||||-------------------------")
    #print(nombre[0].text)
    #print(categoria_especifica[0].text)
    #print(calificacion[0].text)
    #print(reviews[0].text)
    #print(direccion[0].text)
        #print(nombre)
        #print(categoria_especifica)
        #print(calificacion)
        #print(reviews)
        #print(direccion)
        #print(nombre[0].text)
        #print(categoria_especifica[0].text)
        #print(calificacion[0].text)
        #print(reviews[0].text)
        #print(direccion[0].text)
restaux={"Nombre":nombre,"Categoria_Especifica":categoria_especifica,"Calificacion":calificacion,"Reviews":reviews,"Direccion":direccion} #"Url":"https://www.ubereats.com"+restaurante}
        #listarestaurantes.append(restaux)
print(restaux)
